In [95]:
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pylab import rcParams

In [96]:
Oil_Prices = pd.read_csv("Data Historis Minyak Mentah WTI Berjangka.csv")
Oil_Prices.dtypes
Oil_Prices.head()

,Tanggal,Terakhir,Pembukaan,Tertinggi,Terendah,Vol.,Perubahan%
0,01/01/2025,"72,53","71,85","80,77","71,79","6,29M","1,13%"
1,01/12/2024,"71,72","68,00","72,02","66,98","4,65M","5,47%"
2,01/11/2024,"68,00","70,44","72,88","66,61","5,54M","-1,82%"
3,01/10/2024,"69,26","68,41","78,46","66,33","7,35M","1,60%"
4,01/09/2024,"68,17","73,53","74,41","65,27","6,66M","-7,31%"


In [97]:
Oil_Prices.isnull().sum()


Tanggal       0
Terakhir      0
Pembukaan     0
Tertinggi     0
Terendah      0
Vol.          1
Perubahan%    0
dtype: int64

In [98]:
Oil_Prices = Oil_Prices.drop(columns=['Pembukaan', 'Tertinggi', 'Terendah', 'Vol.'])
Oil_Prices['Tanggal'] = pd.to_datetime(Oil_Prices['Tanggal'])
Oil_Prices['Terakhir'] = Oil_Prices['Terakhir'].str.replace(',', '.', regex=False).astype(float)
Oil_Prices['Perubahan%'] = Oil_Prices['Perubahan%'].str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100
Oil_Prices.rename(columns={'Terakhir': 'Harga'}, inplace=True)
Oil_Prices.rename(columns={'Perubahan%': 'Perubahan'}, inplace=True)
Oil_Prices

,Tanggal,Harga,Perubahan
0,2025-01-01,72.53,0.0113
1,2024-01-12,71.72,0.0547
2,2024-01-11,68.00,-0.0182
3,2024-01-10,69.26,0.0160
4,2024-01-09,68.17,-0.0731
...,...,...,...
116,2015-01-05,60.30,0.0112
117,2015-01-04,59.63,0.2527
118,2015-01-03,47.60,-0.0434
119,2015-01-02,49.76,0.0315


In [99]:
Inflation = pd.read_csv("Data Inflasi.csv")
Inflation.dtypes
Inflation.head()

,Periode,Data Inflasi
0,Januari 2025,0.76 %
1,Desember 2024,1.57 %
2,November 2024,1.55 %
3,Oktober 2024,1.71 %
4,September 2024,1.84 %


In [100]:
Inflation.isnull().sum()
Inflation.dtypes

Periode         object
Data Inflasi    object
dtype: object

In [101]:
Inflation['periode_clean'] = Inflation['Periode'].replace({
    'Januari': 'January', 'Februari': 'February', 'Maret': 'March',
    'April': 'April', 'Mei': 'May', 'Juni': 'June', 'Juli': 'July',
    'Agustus': 'August', 'September': 'September', 'Oktober': 'October',
    'November': 'November', 'Desember': 'December'
}, regex=True)

Inflation['Periode'] = pd.to_datetime(Inflation['periode_clean'], format='%B %Y')
Inflation = Inflation.drop(columns=['periode_clean'])
Inflation['Data Inflasi'] = Inflation['Data Inflasi'].str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100
Inflation.rename(columns={'Periode': 'Tanggal'}, inplace=True)

Inflation


,Tanggal,Data Inflasi
0,2025-01-01,0.0076
1,2024-12-01,0.0157
2,2024-11-01,0.0155
3,2024-10-01,0.0171
4,2024-09-01,0.0184
...,...,...
116,2015-05-01,0.0715
117,2015-04-01,0.0679
118,2015-03-01,0.0638
119,2015-02-01,0.0629


In [102]:
Exchange_Rate = pd.read_csv("USD_IDR Historical Data.csv")
Exchange_Rate.dtypes
Exchange_Rate.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,01/01/2025,"16,300.0","16,095.0","16,392.5","16,095.0",NaN,1.31%
1,12/01/2024,"16,090.0","15,870.0","16,322.5","15,830.0",NaN,1.58%
2,11/01/2024,"15,840.0","15,705.0","15,971.5","15,617.5",NaN,0.96%
3,10/01/2024,"15,690.0","15,175.0","15,778.5","15,167.5",NaN,3.67%
4,09/01/2024,"15,135.0","15,497.5","15,583.5","15,065.0",NaN,-2.04%


In [103]:
Exchange_Rate.isnull().sum()

Date          0
Price         0
Open          0
High          0
Low           0
Vol.        121
Change %      0
dtype: int64

In [104]:
Exchange_Rate = Exchange_Rate.drop(columns=['Open', 'High', 'Low', 'Vol.'])
Exchange_Rate['Date'] = pd.to_datetime(Exchange_Rate['Date'])
Exchange_Rate['Price'] = Exchange_Rate['Price'].str.replace(',', '') 
Exchange_Rate['Price'] = Exchange_Rate['Price'].astype(float).astype(int) 
Exchange_Rate['Change %'] = Exchange_Rate['Change %'].str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100
Exchange_Rate.rename(columns={'Price': 'Harga'}, inplace=True)
Exchange_Rate.rename(columns={'Change %': 'Perubahan'}, inplace=True)
Exchange_Rate.rename(columns={'Date': 'Tanggal'}, inplace=True)
Exchange_Rate.dtypes

Tanggal      datetime64[ns]
Harga                 int32
Perubahan           float64
dtype: object

In [105]:
Exchange_Rate

,Tanggal,Harga,Perubahan
0,2025-01-01,16300,0.0131
1,2024-12-01,16090,0.0158
2,2024-11-01,15840,0.0096
3,2024-10-01,15690,0.0367
4,2024-09-01,15135,-0.0204
...,...,...,...
116,2015-05-01,13223,0.0203
117,2015-04-01,12960,-0.0084
118,2015-03-01,13070,0.0116
119,2015-02-01,12920,0.0201


In [106]:
Interest_Rate = pd.read_csv("BI Rate.csv")
Interest_Rate.dtypes
Interest_Rate

,Bulan,BI Rate
0,Jan-15,7.75
1,Feb-15,7.50
2,Mar-15,7.50
3,Apr-15,7.50
4,May-15,7.50
...,...,...
116,Sep-24,6.00
117,Oct-24,6.00
118,Nov-24,6.00
119,Dec-24,6.00


In [107]:
Interest_Rate.isnull().sum()

Bulan      0
BI Rate    0
dtype: int64

In [108]:
Interest_Rate['Bulan'] = pd.to_datetime(Interest_Rate['Bulan'], format='%b-%y')
Interest_Rate.rename(columns={'Bulan': 'Tanggal'}, inplace=True)
Interest_Rate.rename(columns={'BI Rate': 'Interest Rate'}, inplace=True)
Interest_Rate

,Tanggal,Interest Rate
0,2015-01-01,7.75
1,2015-02-01,7.50
2,2015-03-01,7.50
3,2015-04-01,7.50
4,2015-05-01,7.50
...,...,...
116,2024-09-01,6.00
117,2024-10-01,6.00
118,2024-11-01,6.00
119,2024-12-01,6.00


In [109]:
Gross_Domestic_Product = pd.read_csv("gdp.csv")
Gross_Domestic_Product.dtypes
Gross_Domestic_Product

,Tahun,PDB (Milyar),Perubahan (%)
0,2024,"12,920,281.70",5.03
1,2023,"12,301,475.10",5.05
2,2022,"11,710,223.20",5.31
3,2021,"11,120,059.70",3.70
4,2020,"10,722,999.30",-2.07
5,2019,"10,949,155.40",5.02
6,2018,"10,425,851.90",5.17
7,2017,"9,912,928.10",5.07
8,2016,"9,434,613.40",5.03
9,2015,"8,982,517.10",4.88


In [110]:
Gross_Domestic_Product.isnull().sum()

Tahun            0
PDB (Milyar)     0
Perubahan (%)    0
dtype: int64

In [111]:
Gross_Domestic_Product['Tahun'] = pd.to_datetime(Gross_Domestic_Product['Tahun'], format='%Y')
Gross_Domestic_Product['PDB (Milyar)'] = Gross_Domestic_Product['PDB (Milyar)'].str.replace(',', '', regex=False).astype(float)
Gross_Domestic_Product.rename(columns={'Perubahan (%)': 'Perubahan'}, inplace=True)
Gross_Domestic_Product.dtypes

Tahun           datetime64[ns]
PDB (Milyar)           float64
Perubahan              float64
dtype: object

In [112]:
Gross_Domestic_Product

,Tahun,PDB (Milyar),Perubahan
0,2024-01-01,12920281.7,5.03
1,2023-01-01,12301475.1,5.05
2,2022-01-01,11710223.2,5.31
3,2021-01-01,11120059.7,3.70
4,2020-01-01,10722999.3,-2.07
5,2019-01-01,10949155.4,5.02
6,2018-01-01,10425851.9,5.17
7,2017-01-01,9912928.1,5.07
8,2016-01-01,9434613.4,5.03
9,2015-01-01,8982517.1,4.88


In [113]:
Gold_Prices = pd.read_csv("Data Historis GAU_IDR.csv")
Gold_Prices.dtypes

Tanggal        object
Terakhir       object
Pembukaan      object
Tertinggi      object
Terendah       object
Vol.          float64
Perubahan%     object
dtype: object

In [114]:
Gold_Prices.isnull().sum()

Tanggal        0
Terakhir       0
Pembukaan      0
Tertinggi      0
Terendah       0
Vol.          52
Perubahan%     0
dtype: int64

In [115]:
Gold_Prices = Gold_Prices.drop(columns=['Pembukaan', 'Tertinggi', 'Terendah', 'Vol.'])
Gold_Prices['Tanggal'] = pd.to_datetime(Gold_Prices['Tanggal'])
Gold_Prices['Terakhir'] = Gold_Prices['Terakhir'].str.replace('.', '', regex=False).astype(int)
Gold_Prices['Perubahan%'] = Gold_Prices['Perubahan%'].str.replace(',', '.', regex=False).str.replace('%', '', regex=False).astype(float) / 100
Gold_Prices.rename(columns={'Terakhir': 'Harga'}, inplace=True)
Gold_Prices.rename(columns={'Perubahan%': 'Perubahan'}, inplace=True)
Gold_Prices

,Tanggal,Harga,Perubahan
0,2025-01-01,1465571,0.0795
1,2024-01-12,1357613,0.0048
2,2024-01-11,1351090,-0.0270
3,2024-01-10,1388652,0.0778
4,2024-01-09,1288365,0.0306
5,2024-01-08,1250141,-0.0224
6,2024-01-07,1278844,0.0463
7,2024-01-06,1222195,0.0049
8,2024-01-05,1216236,0.0146
9,2024-01-04,1198769,0.0475
